In [58]:
# These are all the modules we'll be using later. Make sure you can
# import them before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [59]:
import os
data_root = '.'
created_pickle_file = os.path.join(data_root, 'notMNIST.pickle')
with open(created_pickle_file, 'rb') as f:
    read_data = pickle.load(f)

train_ds = read_data['train_dataset']
train_lbls = read_data['train_labels']
print(train_ds.shape, train_lbls.shape)

test_ds = read_data['test_dataset']
test_lbls = read_data['test_labels']
print(test_ds.shape, test_lbls.shape)

validn_ds = read_data['validn_dataset']
validn_lbls = read_data['validn_labels']
print(validn_ds.shape, validn_lbls.shape)

(20000, 28, 28) (20000,)
(1000, 28, 28) (1000,)
(10000, 28, 28) (10000,)


In [60]:
# now we are converting data to our deisre format
# Nx26x26 -> Nx784 for the datapoitns
# Nx1 (1 to 10) - >Nx10 (one hot encoding)
num_labels = 10
def reformat(ds, lbls):
    n, w, h = ds.shape
    ds = ds.reshape((-1, w*h)).astype(np.float32)
    lbls = (np.arange(num_labels)==lbls[:,None]).astype(np.float32)
    return ds, lbls

tr_ds, tr_lb = reformat(train_ds, train_lbls)
ts_ds, ts_lb = reformat(test_ds, test_lbls)
vl_ds, vl_lb = reformat(validn_ds, validn_lbls)

print(tr_ds.shape, tr_lb.shape)
print(ts_ds.shape, ts_lb.shape)
print(vl_ds.shape, vl_lb.shape)

(20000, 784) (20000, 10)
(1000, 784) (1000, 10)
(10000, 784) (10000, 10)


In [61]:
# We first create a graph explaining our computation.
# All these inputs, ops and the operations.
# These get created as nodes over computation graph

In [62]:
train_subset = 10000

graph = tf.Graph()
with graph.as_default():
    # input - load all thee datasets as constants
    tf_tr_ds = tf.constant(tr_ds[:train_subset, :]) # only this much row
    tf_tr_lb = tf.constant(tr_lb[:train_subset, :])
    
    tf_ts_ds = tf.constant(ts_ds)
    tf_ts_lb = tf.constant(ts_lb)
    
    tf_vl_ds = tf.constant(vl_ds)
    tf_vl_lb = tf.constant(vl_lb)
    
    
    # variables
    rows, features = tr_ds.shape
    # we basically train a grad descent for each label.
    # that is why we have 
    weights = tf.Variable(tf.truncated_normal([features, num_labels]))
    biases = tf.Variable(tf.zeros(num_labels))
    
    
    # training computation
    
    logits = tf.matmul(tf_tr_ds, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_tr_lb, logits=logits))
    
    
    # optimizer
    opt = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    
    # final predictions pipeline
    train_pred = tf.nn.softmax(logits)
    valid_pred = tf.nn.softmax(tf.matmul(tf_vl_ds, weights) + biases)
    test_pred = tf.nn.softmax(tf.matmul(tf_ts_ds, weights) + biases)
    
    
    

In [63]:
num_steps = 1100
def acc(pred, labels):
    return (100.0* np.sum(np.argmax(pred, 1) == np.argmax(labels, 1))
            /pred.shape[0] )

In [64]:
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('init')
    for step in range(num_steps):
        _, l, pred = session.run([opt, loss, train_pred])
        if(step%100 == 0):
            print ('\nloss at step %d: %f\n' %(step, l))
            print ('training accuracy: %f' %(acc(pred, tr_lb[:train_subset, :])))
            print ('validation accuracy: %f' %(acc(valid_pred.eval(), vl_lb)))
            print ('test accuracy: %f' %(acc(test_pred.eval(), ts_lb)))
        

init

loss at step 0: 18.055855

training accuracy: 10.130000
validation accuracy: 12.570000
test accuracy: 12.500000

loss at step 100: 2.480109

training accuracy: 71.350000
validation accuracy: 70.130000
test accuracy: 78.700000

loss at step 200: 1.982196

training accuracy: 74.480000
validation accuracy: 72.210000
test accuracy: 81.400000

loss at step 300: 1.715235

training accuracy: 75.970000
validation accuracy: 73.110000
test accuracy: 82.500000

loss at step 400: 1.536018

training accuracy: 76.860000
validation accuracy: 73.650000
test accuracy: 82.700000

loss at step 500: 1.402711

training accuracy: 77.370000
validation accuracy: 74.190000
test accuracy: 83.200000

loss at step 600: 1.297624

training accuracy: 78.050000
validation accuracy: 74.460000
test accuracy: 83.900000

loss at step 700: 1.211856

training accuracy: 78.620000
validation accuracy: 74.420000
test accuracy: 84.100000

loss at step 800: 1.140198

training accuracy: 78.860000
validation accuracy: 74.68

In [65]:
batch_size = 128
graph = tf.Graph()
n, w, h = train_ds.shape
with graph.as_default():
    # since we are changing our mini batch every iteration, we are making them 
    # variables.
    tf_tr_ds = tf.placeholder(tf.float32, shape=(batch_size, w*h))
    tf_tr_lbls = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    # this is used only for testing. So constants only.
    tf_vl_ds = tf.constant(vl_ds)
    tf_ts_ds = tf.constant(ts_ds)
    
    # weights and biases as usual
    weights = tf.Variable(tf.truncated_normal([w*h, num_labels]))
    biases = tf.Variable(tf.zeros(num_labels))
    
    # training
    logits = tf.matmul(tf_tr_ds, weights) + biases
    print (tf.matmul(tf_tr_ds, weights))
    print (biases)
    print (logits)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_tr_lbls, logits=logits))
    
    
    #optimizer
    opt = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    
    #preds
    tr_pred = tf.nn.softmax(logits) # Step 3
    
    vl_pred = tf.nn.softmax(tf.matmul(tf_vl_ds, weights)+biases)
    ts_pred = tf.nn.softmax(tf.matmul(tf_ts_ds, weights)+biases)
    

Tensor("MatMul_1:0", shape=(128, 10), dtype=float32)
<tf.Variable 'Variable_1:0' shape=(10,) dtype=float32_ref>
Tensor("add:0", shape=(128, 10), dtype=float32)


In [66]:
num_steps = 3001
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print('init')
    for step in range(num_steps):
        offset = (step*batch_size)%(tr_lb.shape[0] - batch_size)
        # minibatch
        batch_data = tr_ds[offset:(offset+batch_size), :]
        batch_labels = tr_lb[offset:(offset+batch_size), :]
        
        # since it is a placeholder, we have to create a dictionary to feed valeus in
        feed_dict={tf_tr_ds : batch_data, tf_tr_lbls : batch_labels}
        _, l, preds = sess.run([opt, loss, tr_pred], feed_dict=feed_dict) #all steps
        if(step%500==0):
            print("minibatch loss at step %d:%f \n" %(step, l))
            print("minibatch accuracy : %f" %acc(preds, batch_labels))
            print("validn accuracy :%f" %acc(vl_pred.eval(), vl_lb) )
            print("test accuracy :%f" %acc(ts_pred.eval(), ts_lb) )

init
minibatch loss at step 0:26.144901 
minibatch accuracy : 3.906250
validn accuracy :10.200000
test accuracy :8.300000
minibatch loss at step 500:1.614322 
minibatch accuracy : 74.218750
validn accuracy :74.610000
test accuracy :83.200000
minibatch loss at step 1000:1.323085 
minibatch accuracy : 76.562500
validn accuracy :75.490000
test accuracy :84.000000
minibatch loss at step 1500:1.039775 
minibatch accuracy : 82.812500
validn accuracy :75.670000
test accuracy :84.800000
minibatch loss at step 2000:1.022939 
minibatch accuracy : 80.468750
validn accuracy :76.690000
test accuracy :84.900000
minibatch loss at step 2500:0.980337 
minibatch accuracy : 78.125000
validn accuracy :76.940000
test accuracy :85.100000
minibatch loss at step 3000:0.921970 
minibatch accuracy : 82.031250
validn accuracy :76.890000
test accuracy :86.400000


In [ ]:
batch_size = 128
graph = tf.Graph()
n, w, h = train_ds.shape
num_hidden_nodes = 1024
with graph.as_default():
    # since we are changing our mini batch every iteration, we are making them 
    # variables.
    tf_tr_ds = tf.placeholder(tf.float32, shape=(batch_size, w*h))
    tf_tr_lbls = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    # this is used only for testing. So constants only.
    tf_ts_ds = tf.constant(ts_ds)
    tf_ts_lb = tf.constant(ts_lb)
    
    tf_vl_ds = tf.constant(vl_ds)
    tf_vl_lb = tf.constant(vl_lb)
    
    # weights and biases  - input layer and hidden layer
    weights1 = tf.Variable(tf.truncated_normal([w*h, num_hidden_nodes]))
    biases1 = tf.Variable(tf.zeros(num_hidden_nodes))
    
    # pre activation of hidden layer
    logits1 = tf.matmul(tf_tr_ds, weights1) + biases1

    # post activation of hidden layer
    relu_op = tf.nn.relu(logits1)
    
    # weights and biases  - hidden layer and output layer
    weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros(num_labels))
    
    # pre aactivation of output layer
    logits2 = tf.matmul(relu_op, weights2) + biases2
    
    # training
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_tr_lbls, logits=logits2))
    
    
    #optimizer
    opt = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    
    #preds
    tr_pred = tf.nn.softmax(logits2) # Step 3
    vl_pred = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_vl_ds, weights1)+biases1), weights2) + biases2)
    ts_pred = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_ts_ds, weights1)+biases1), weights2) + biases2)
    

In [ ]:
num_steps = 3001
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print('init')
    for step in range(num_steps):
        offset = (step*batch_size)%(tr_lb.shape[0] - batch_size)
        # minibatch
        batch_data = tr_ds[offset:(offset+batch_size), :]
        batch_labels = tr_lb[offset:(offset+batch_size), :]
        
        # since it is a placeholder, we have to create a dictionary to feed valeus in
        feed_dict={tf_tr_ds : batch_data, tf_tr_lbls : batch_labels}
        _, l, preds = sess.run([opt, loss, tr_pred], feed_dict=feed_dict) #all steps
        if(step%500==0):
            print("minibatch loss at step %d:%f \n" %(step, l))
            print("minibatch accuracy : %f" %acc(preds, batch_labels))
            print("validn accuracy :%f" %acc(vl_pred.eval(), vl_lb) )
            print("test accuracy :%f" %acc(ts_pred.eval(), ts_lb) )

init
minibatch loss at step 0:349.950409 
minibatch accuracy : 15.625000
validn accuracy :22.870000
test accuracy :24.100000
minibatch loss at step 500:8.415914 
minibatch accuracy : 88.281250
validn accuracy :80.680000
test accuracy :88.700000
minibatch loss at step 1000:0.314708 
minibatch accuracy : 96.093750
validn accuracy :82.540000
test accuracy :89.700000
minibatch loss at step 1500:1.093589 
minibatch accuracy : 94.531250
validn accuracy :81.900000
test accuracy :90.300000
